# Heterologous pathway addition and importing the model

## importing the model

In [2]:
from cobra.io import read_sbml_model, write_sbml_model
model = read_sbml_model('/Users/wagner/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM.xml') # loading
from cobra import Reaction, Metabolite, Gene



In [47]:
?write_sbml_model

Signature:
write_sbml_model(
    cobra_model: cobra.core.model.Model,
    filename: Union[str, IO, pathlib.Path],
    f_replace: dict = {'F_GENE': <function _f_gene at 0x16b2e36a0>, 'F_GENE_REV': <function _f_gene_rev at 0x16b2e3880>, 'F_SPECIE': <function _f_specie at 0x16b2e3920>, 'F_SPECIE_REV': <function _f_specie_rev at 0x16b2e39c0>, 'F_REACTION': <function _f_reaction at 0x16b2e3a60>, 'F_REACTION_REV': <function _f_reaction_rev at 0x16b2e3b00>, 'F_GROUP': <function _f_group at 0x16b2e3ba0>, 'F_GROUP_REV': <function _f_group_rev at 0x16b2e3c40>},
    **kwargs,
) -> None
Docstring:
Write cobra model to filename.

The created model is SBML level 3 version 1 (L1V3) with
fbc package v2 (fbc-v2).

If the given filename ends with the suffix ".gz" (for example,
"myfile.xml.gz"), libSBML assumes the caller wants the file to be
written compressed in gzip format. Similarly, if the given filename
ends with ".zip" or ".bz2", libSBML assumes the caller wants the
file to be compressed in zip or

In [3]:
model

Name,yeastGEM_v8__46__7__46__0
Memory address,11e79eb90
Number of metabolites,2807
Number of reactions,4133
Number of genes,1163
Number of groups,106
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


Since there are no production pathways inherently in the model of the organism we will need to add the metabolite and reaction ourselves

Adding the metabolites:

In [4]:
Beta_farnesene = Metabolite(name='Beta-farnesene',id='beta_farnesene_c',compartment='c')
# Created the metabolite Beta-farnesene as a metabolite that is present in the cytoplasm

Adding the reaction for beta-farnesene and the demand reaction for beta-farnesene

In [5]:
from cobra import Reaction, Metabolite, Gene

Beta_farnesene_synthase_reaction = Reaction('R08695') # Creates the Beta-farnesene reaction 

# This creates the reaction for beta-farnesene synthesis by indicating the consumption of the substrates and 
# using our newly introduced metabolite as the product
# The reaction R08695 is:
# Trans,trans-Farnesyl diphosphate <=> beta-Farnesene + Diphosphate
# These are found as the metabolites:
# Farnesyl diphosphate = s_0190
# beta-farnesene = Beta_farnesene
# Diphosphate = s_0633 

# Thus having already defined the metabolite and the reaction we can fully define the reaction stoichiometry:

Beta_farnesene_synthase_reaction.add_metabolites({model.metabolites.s_0190: -1,
                                                  model.metabolites.s_0633: 1,
                                                  Beta_farnesene: 1 # our product
                                                  }) 

# Test if the reaction has been made properly and the stoichiometric definition is correct:
Beta_farnesene_synthase_reaction

# Adding the reaction to the model
model.add_reactions([Beta_farnesene_synthase_reaction])

# tests that the reaction has been added to the model
model.reactions.get_by_id(Beta_farnesene_synthase_reaction.id)

Ignoring reaction 'R08695' since it already exists.


Reaction identifier,R08695
Name,
Memory address,0x29995d090
Stoichiometry,s_0190 --> beta_farnesene_c + s_0633 farnesyl diphosphate --> Beta-farnesene + diphosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
# This creates the demand reaction for beta-farnesene as this otherwise would become a dead-end metabolite:

# The demand reaction follows the same framework as above, with the stoichiometry just being -1 for the
# metabolite Beta-farnesene:

Beta_farnesene_demand_reaction = Reaction('BFD')
Beta_farnesene_demand_reaction.add_metabolites({
                                                  Beta_farnesene: -1 # our product
                                                  })
model.add_reactions([Beta_farnesene_demand_reaction])

# Note: the output is because we already added the reaction to the model.

Ignoring reaction 'BFD' since it already exists.


In [7]:
# tests that the reaction has been added to the model
model.reactions.get_by_id(Beta_farnesene_demand_reaction.id)



Reaction identifier,BFD
Name,
Memory address,0x29995d210
Stoichiometry,beta_farnesene_c --> Beta-farnesene -->
GPR,
Lower bound,0.0
Upper bound,1000.0


## Testing to make sure flux goes through the production pathway that we have created

In [8]:
# This function test to see that the organism still is growing
with model:
   print('Maximal growth rate',model.optimize().objective_value)

# now we will set the objective function of the model to the demand reaction for beta-farnesene to test if
# the model allows flux through the beta-farnsene biosynthesis pathway
with model:
   model.objective = model.reactions.BFD
   print('This is the maximal production of beta-farnesene on glucose:', model.optimize().objective_value, 'mmol beta-farnesene gDW^-1 h^-1')

Maximal growth rate 0.08192817234184746
This is the maximal production of beta-farnesene on glucose: 0.2303511221971975 mmol beta-farnesene gDW^-1 h^-1


It is evident that we are seing flux through the reaction when setting it as the model objective function, this means that it is not a dead end metabolite.

# Saving the model:

In [9]:
# This code saves the changes to the model and we now have a model that has a the heterologous pathway for 
# beta-farnesene production as well as a demand reaction for cytoplasmic beta-farnesene

write_sbml_model(model,
                 filename = '/Users/wagner/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM2.xml'
                 )  
#saving the model